In [5]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, pipeline

In [2]:
model=TFGPT2LMHeadModel.from_pretrained("ai_model/dialogue_generation/dialogue_model")
tokenizer=GPT2Tokenizer.from_pretrained("ai_model/dialogue_generation/dialogue_tokenizer")

2025-05-24 23:40:20.516787: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-24 23:40:20.676579: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-24 23:40:20.676650: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-24 23:40:20.680038: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-24 23:40:20.680083: I external/local_xla/xla/stream_executor

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [12]:
def generate_dialogue(prompt, max_length=50, temperature=0.7, top_k=50):
    # Encode the prompt
    inputs = tokenizer(
        prompt,
        return_tensors="tf",
        padding=True,
        truncation=True,
        max_length=128
    )
    
    # Generate response
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_return_sequences=1,
        temperature=temperature,  # Control randomness
        top_k=top_k,             # Limit to top-k probable tokens
        do_sample=True,          # Enable sampling for varied responses
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated tokens
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [13]:
# Test the model
text = "I'm feeling a bit down today. Can you cheer me up?"
response = generate_dialogue(text)
print("Input:", text)
print("Response:", response)

Input: I'm feeling a bit down today. Can you cheer me up?
Response: I'm feeling a bit down today. Can you cheer me up?????????????????????????????????????


In [14]:
text = "Context: I just got a promotion at work! I'm so excited!\nResponse:"
response = generate_dialogue(text)
print("Input:", text)
print("Response:", response)

Input: Context: I just got a promotion at work! I'm so excited!
Response:
Response: Context: I just got a promotion at work! I'm so excited!
Response:::::::::::::::::::::::::::::::::


In [19]:
import pandas as pd
import json

# Load training history
history = pd.read_csv('data/dialogue_training_history.csv')

# Create line chart for loss and perplexity
chart_config = {
    "type": "line",
    "data": {
        "labels": list(range(1, len(history) + 1)),
        "datasets": [
            {
                "label": "Training Loss",
                "data": history['loss'].tolist(),
                "borderColor": "#1f77b4",
                "fill": False
            },
            {
                "label": "Validation Loss",
                "data": history['val_loss'].tolist(),
                "borderColor": "#ff7f0e",
                "fill": False
            },
            {
                "label": "Validation Perplexity",
                "data": history['val_perplexity'].tolist(),
                "borderColor": "#2ca02c",
                "fill": False,
                "yAxisID": "y2"
            }
        ]
    },
    "options": {
        "scales": {
            "y": {
                "title": {"display": True, "text": "Loss"},
                "beginAtZero": True
            },
            "y2": {
                "position": "right",
                "title": {"display": True, "text": "Perplexity"},
                "beginAtZero": True
            },
            "x": {
                "title": {"display": True, "text": "Epoch"}
            }
        },
        "plugins": {
            "title": {"display": True, "text": "Training and Validation Metrics"}
        }
    }
}


# Running the chart generation
chart_json = json.dumps(chart_config)

# DIsplay the chart configuration
print("Chart Configuration JSON:")
print(chart_json)

Chart Configuration JSON:
{"type": "line", "data": {"labels": [1], "datasets": [{"label": "Training Loss", "data": [0.0063034580089151], "borderColor": "#1f77b4", "fill": false}, {"label": "Validation Loss", "data": [7.63030220696237e-06], "borderColor": "#ff7f0e", "fill": false}, {"label": "Validation Perplexity", "data": [1.0000076291434103], "borderColor": "#2ca02c", "fill": false, "yAxisID": "y2"}]}, "options": {"scales": {"y": {"title": {"display": true, "text": "Loss"}, "beginAtZero": true}, "y2": {"position": "right", "title": {"display": true, "text": "Perplexity"}, "beginAtZero": true}, "x": {"title": {"display": true, "text": "Epoch"}}}, "plugins": {"title": {"display": true, "text": "Training and Validation Metrics"}}}}
